In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('float_format', '{:0.3f}'.format)

In [3]:
housesDf = pd.read_csv(r'kc_house_data.csv')

housesDf['living_price'] = housesDf['price'] / housesDf['sqft_living']
housesDf['lot_price'] = housesDf['price'] / housesDf['sqft_lot']

In [4]:
housesFiltered = housesDf.filter(['price', 'bedrooms', 'bathrooms', 'floors', 'view', 'condition', 'yr_built', 'living_price', 'lot_price','grade'])

In [5]:
housesGrades = housesFiltered['grade']
meansByGradeDf = pd.DataFrame()

for i in range(housesGrades.min(), housesGrades.max() + 1):
    meansByGradeDf[f'{i}'] = housesFiltered[housesDf['grade'] == i].mean(numeric_only=True)

meansByGradeDf = meansByGradeDf.T
meansByGradeDf

,price,bedrooms,bathrooms,floors,view,condition,yr_built,living_price,lot_price,grade
1,142000.000,0.000,0.000,1.000,0.000,1.000,1963.000,489.655,6.802,1.000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,205666.667,1.000,0.250,1.000,0.000,3.333,1945.333,360.818,11.113,3.000
4,214381.034,1.483,0.905,1.034,0.138,3.103,1939.138,362.381,26.450,4.000
5,248523.971,2.388,1.114,1.087,0.136,3.492,1935.455,278.934,35.438,5.000
6,301919.637,2.686,1.243,1.109,0.093,3.540,1942.471,270.992,48.601,6.000
7,402590.262,3.256,1.829,1.297,0.100,3.491,1963.627,251.902,71.489,7.000
8,542852.766,3.480,2.348,1.668,0.247,3.355,1980.400,258.087,113.892,8.000
9,773513.186,3.773,2.664,1.849,0.429,3.266,1988.419,275.767,115.660,9.000
10,1071771.074,3.907,3.006,1.893,0.654,3.233,1989.570,307.589,107.043,10.000


In [6]:
mainMeans = pd.DataFrame()
housesGradesMean = np.mean(housesGrades)

mainMeans['mean_grade_hi_7'] = meansByGradeDf[meansByGradeDf['grade'] >= housesGradesMean].mean()
mainMeans['std'] = housesFiltered.std(ddof=1)
mainMeans['corr_with_grade'] = housesFiltered.corr()['grade']

mainMeans.drop('grade', axis=0)

mainMeans = mainMeans.T
mainMeans

,price,bedrooms,bathrooms,floors,view,condition,yr_built,living_price,lot_price,grade
mean_grade_hi_7,1630969.356,4.065,3.433,1.880,0.971,3.236,1986.588,344.866,118.688,10.500
std,367127.196,0.930,0.770,0.540,0.766,0.651,29.373,110.061,94.055,1.175
corr_with_grade,0.667,0.357,0.665,0.458,0.251,-0.145,0.447,0.123,0.218,1.000


#
## Filtrando pelas casas acima das métricas comuns

In [7]:
housesDf[
    (housesDf['price'] < (mainMeans['price']['mean_grade_hi_7'])) &
    (housesDf['bedrooms'] >= round(mainMeans['bedrooms']['mean_grade_hi_7'] - mainMeans['bedrooms']['std'])) &
    (housesDf['bathrooms'] >= round(mainMeans['bathrooms']['mean_grade_hi_7'] + mainMeans['bathrooms']['std'])) &
    (housesDf['condition'] >= round(mainMeans['condition']['mean_grade_hi_7'] + mainMeans['condition']['std'])) &
    (housesDf['yr_built'] >= round(mainMeans['yr_built']['mean_grade_hi_7']))
].sort_values(['lot_price'], ascending=True).head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,living_price,lot_price
7280,922059169,20141201T000000,800000.000,6,4.250,5480,189050,2.000,0,0,...,340,1991,0,98031,47.412,-122.168,2470,10429,145.985,4.232
11685,1126069045,20140620T000000,1135000.000,6,4.250,6900,244716,2.000,0,0,...,2080,2002,0,98077,47.751,-122.012,4170,266587,164.493,4.638
13403,524069020,20150422T000000,1050000.000,4,4.000,4550,54013,1.000,0,1,...,2250,1989,0,98075,47.596,-122.077,3540,39634,230.769,19.440
9603,1926069192,20140509T000000,1157200.000,4,4.250,5860,52889,2.000,0,0,...,950,1996,0,98072,47.724,-122.095,3320,39066,197.474,21.880
270,4054500390,20141007T000000,1365000.000,4,4.750,5310,57346,2.000,0,0,...,0,1989,0,98077,47.728,-122.042,4180,47443,257.062,23.803
